## groupBy()

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [3]:
schema = StructType([
    StructField('class_cd', StringType()),
    StructField('school', StringType()),
    StructField('class_std_cnt', IntegerType()),
    StructField('loc', StringType()),
    StructField('school_type', StringType()),
    StructField('teaching_type', StringType())
])

cdf = spark.read.csv('/dataframe/a_class_info.csv', header=True, schema=schema)
cdf.printSchema()
cdf.createOrReplaceTempView('class')

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



groupBy  : 집계함수를 가지고 있는 GroupData 객체를 반환한다.  

GrouopData객체의 집계함수들을 사용해 grouping 된 데이터들의 집계결과를 저장하고 있는 DataFrame을 반환 받을 수 있다.

In [30]:
# 지역별 교육타입별 학생 숫자를 구해보자.
cdf.groupBy(cdf.loc, cdf.teaching_type) \
    .agg(sum(cdf.class_std_cnt)) \
    .show()

print('''지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')
print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')
cdf.groupBy(cdf.loc, cdf.teaching_type) \
    .agg(sum(cdf.class_std_cnt).alias('total'), avg('class_std_cnt').alias('avg')) \
    .where(col('total') >= 300) \
    .show()

# .where(sum(cdf.class_std_cnt) >= 300)# 이렇게 해도 되지만 
# 이미 agg함수를 통해 함계를 구해서 새로운 데이터 프레임을 만들었는데
# where절에서 기존 cdf의 class_std_cnt의 합계를 다시 구해서 조건을 행한다.
# 그래서 새로 만들어진 데이터프레임의 컬럼을 반환해주는 
# col()함수를 사용하는게 더 시스템적으로 좋다.


+--------+-------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|
+--------+-------------+------------------+
|   Rural| Experimental|               211|
|    null|         null|              null|
|   Urban|     Standard|               631|
|Suburban|     Standard|               433|
|   Rural|     Standard|               327|
|Suburban| Experimental|               284|
|   Urban| Experimental|               275|
+--------+-------------+------------------+

지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.
컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자
+----------+
|min(total)|
+----------+
|       211|
+----------+



In [97]:
# 학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자
# cdf.groupBy(cdf.loc) \
#     .agg(count(cdf.school).alias('cnt'),sum(cdf.class_std_cnt).alias('stu')) \
#     .show()

# cdf.groupBy(cdf.loc) \
#     .agg(count(cdf.school).alias('cnt'),sum(cdf.class_std_cnt).alias('stu')) \
#     .agg(max(col('stu'))-min(col('stu'))) \
#     .show()

############################################
table = cdf.where(cdf.loc.isNotNull()) \
            .groupBy(cdf.loc) \
            .agg(count(cdf.school).alias('cnt'),sum(cdf.class_std_cnt).alias('stu')) 
table.show()

min_max = base.select(max('cnt'),min('cnt')).collect()
min_max
# min = min_max.collect()[0][0]
# max = min_max.collect()[0][1]

# table.where(table.cnt == min and table.cnt == max).show()

+--------+---+---+
|     loc|cnt|stu|
+--------+---+---+
|   Urban| 37|906|
|Suburban| 34|717|
|   Rural| 26|538|
+--------+---+---+



TypeError: 'int' object is not callable

In [75]:
# 반이 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자
cdf.printSchema()
base = cdf \
            .where(cdf.loc.isNotNull()) \
            .groupBy(cdf.loc) \
            .agg(count(cdf.class_cd).alias('cnt'), sum(cdf.class_std_cnt).alias('tot')) 
base.show()

min_max_row = base.select(max('cnt'),min('cnt')).collect()
min_max_row

base.where(base.cnt.isin(min_max_row[0][0],min_max_row[0][1])) \
    .select(max(col('tot'))-min(col('tot'))) \
    .show()


root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

+--------+---+---+
|     loc|cnt|tot|
+--------+---+---+
|   Urban| 37|906|
|Suburban| 34|717|
|   Rural| 28|538|
+--------+---+---+



[Row(max(cnt)=37, min(cnt)=28)]

+---------------------+
|(max(tot) - min(tot))|
+---------------------+
|                  368|
+---------------------+



#### sql

In [7]:
cdf.printSchema()
# 지역별 교육타입별 학생 숫자를 구해보자.
spark.sql('''
    select loc, teaching_type, sum(class_std_cnt)
    from class
    group by loc, teaching_type
''').show()

print('''지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')
print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')
spark.sql('''
    select loc, teaching_type, sum(class_std_cnt) as total , avg(class_std_cnt) as avg
    from class
    group by loc, teaching_type
    having sum(class_std_cnt) >= 300
''').show()





root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



+--------+-------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|
+--------+-------------+------------------+
|   Rural| Experimental|               211|
|    null|         null|              null|
|   Urban|     Standard|               631|
|Suburban|     Standard|               433|
|   Rural|     Standard|               327|
|Suburban| Experimental|               284|
|   Urban| Experimental|               275|
+--------+-------------+------------------+

지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.
컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자


+--------+-------------+-----+-----------------+
|     loc|teaching_type|total|              avg|
+--------+-------------+-----+-----------------+
|   Urban|     Standard|  631|24.26923076923077|
|Suburban|     Standard|  433|            21.65|
|   Rural|     Standard|  327|          20.4375|
+--------+-------------+-----+-----------------+



## orderBy()

In [17]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
cdf.select('*').orderBy(cdf.class_std_cnt, ascending=False).show(3)
cdf.select('*').orderBy(cdf.class_std_cnt.desc()).show(3)

print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
cdf.select('*').orderBy([cdf.loc,cdf.school], ascending=[0,1]).show(3) 
cdf.orderBy(cdf.loc.asc(), cdf.school.desc()) # select('*') 생략가능

print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
cdf.select('*').orderBy(cdf.school_type.asc_nulls_first()).show(3)
cdf.select('*').orderBy(cdf.school_type.asc_nulls_last()).show(3)

반 학생 숫자를 기준으로 내림차순 정렬하라
+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     18K| GOOBU|           31|Urban|     Public|     Standard|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+
only showing top 3 rows

+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     18K| GOOBU|           31|Urban|     Public|     Standard|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+
only showing top 3 rows

loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼

#### sql

In [8]:
cdf.printSchema()
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
spark.sql('''
    select * from class order by class_std_cnt desc
''').show(3)


print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
spark.sql('''
    select * from class order by loc asc, school desc
''').show(3)


print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
spark.sql('''
    select * from class order by school_type nulls first
''').show(3)
spark.sql('''
    select * from class order by school_type nulls last
''').show(3)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)

반 학생 숫자를 기준으로 내림차순 정렬하라
+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     18K| GOOBU|           31|Urban|     Public|     Standard|
|     Q0E| ZOWMK|           30|Urban|     Public| Experimental|
|     YTB| VVTVA|           30|Urban|     Public| Experimental|
+--------+------+-------------+-----+-----------+-------------+
only showing top 3 rows

loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라
+--------+------+-------------+----+-----------+-------------+
|class_cd|school|class_std_cnt| loc|school_type|teaching_type|
+--------+------+-------------+----+-----------+-------------+
|     5